# Para executar este programa pressione Crtl+Enter dentro de cada célula do código ou clicando no símbolo ►.

**1. Apenas rodar. Em caso de várias geocodificações só é necessário rodar uma vez. (Importa e instala bibliotecas).**

In [ ]:
!pip install geopandas 
!pip install geopy
import geopandas as gpd
import pandas as pd
import geopy as gpy
from tqdm import tqdm
from geopy.extra.rate_limiter import RateLimiter

**2. Importar csv(; - utf-8). Clicar em browser, selecionar o arquivo, esperar o upload.(Importa csv delimitador por ';' com endereços).**

In [ ]:
from google.colab import files
uploaded = files.upload()

**3. Substituir ['exemplo.csv'] dentro de colchetes pelo nome do arquivo csv. Manter aspas e colchetes.(Armazena csv em variável).**

In [49]:
import io
x = pd.read_csv('exemplo.csv',sep=';')

**4. Substituir ['Endereco_completo'] dentro de colchetes pelo nome da coluna correspondente no arquivo csv.**

> obs.: Nesta etapa é feita a escolha de qual servidor será utilizado para geodificação. Este programa está pré-configurado para utilizar o HERE, o qual necessita de uma chave (API key) que pode ser obtida através de uma licença trial [clicando aqui](https://developer.here.com/). Substitua o 'app_id' e o 'app_code' no código pela api obtida. Para utilizar outros servidores confira as notas no final programa.








In [ ]:
from geopy.geocoders import Here
geolocator = Here(user_agent='geoapp', timeout = 86400, app_id = 'pds9Pf6AI0F3FbVNRePx', app_code='UZs_DzoE6PAJwYE5ZUj4uw')#substituir app_id e app_code

geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1/20,max_retries = 100, error_wait_seconds=15)
tqdm.pandas()
#substiuir 'Endereco_completo' na linha abaixo pelo nome da coluna correspondente no arquivo csv original. 
x['ENCONTRADO'] = x['Endereco_completo'].progress_apply(geocode)
x['PONTO'] = x['ENCONTRADO'].apply(lambda loc: tuple(loc.point) if loc else None)

**5. Apenas rodar ou substituir 'resultado.csv' pelo nome de arquivo de saída desejado. Para baixar o resultado da geocodificação clique símbolo de 'pasta' ao lado esquerdo. A coluna com as coordenadas será devolvida em formato wkt, podendo ser facilmente espacilizada em softwares como QGIS.** 

 ![texto alternativo](https://uploaddeimagens.com.br/images/002/742/380/thumb/tempsnip.png?1593624873)

**(Adiciona resultado da geocodificação na guia de arquivos ao lado esquerdo).**

In [51]:
#Defini a coluna chave-primária como primeira coluna, excluindo index padrão
#caso sejam iniciadas mais de uma geocoficação sem fechar o navegar é necessário
#excluir a primeira linha do código(linha abaixo).
x.set_index('chave', inplace = True)
x['geo_y'] = x['PONTO'].astype('str').apply(lambda x : x.replace('(',',').split(',')[1])
x['geo_x'] = x['PONTO'].astype('str').apply(lambda x : x.replace('(',',').split(',')[2])
x.drop('PONTO',axis=1,inplace=True)
#substituir 'resultado.csv' na linha abaixo por nome de preferência.
x.to_csv('resultado.csv',sep=';')